In [1]:
# CNN model for CIFAR-10
import numpy
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.losses import mse, logcosh, binary_crossentropy, categorical_crossentropy
from keras.activations import relu, elu, softmax, sigmoid, linear
from keras.constraints import maxnorm
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Reshape
from keras.layers import TimeDistributed, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import TensorBoard
import random
import talos

Using TensorFlow backend.


In [2]:
K.set_image_dim_ordering('tf')

# I. Parametrage des hyper-paramètres

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:
lr = numpy.random.random() * (0.1 - 0.0001) + 0.0001
momentum = numpy.random.random() * (0.1 - 0.0001) + 0.0001
epochs = 1
decay = lr / epochs

In [5]:
Param = {'input_shape': 3072,
             'input_shape_rnn': (32, 96),
             'input_shape_cnn': (32, 32, 3),
             'lr': lr,
             'hidden_dim': 128,
             'units': 512,
             'unitsSlp': 10,
             'last_units': 10,
             'first_neuron': [4, 8, 16, 32, 64],
             'hidden_layers': [2, 4, 6, 8, 9, 10, 20, 25, 30],
             'kernel_constraint': maxnorm(3),
             'batch_size': (64, 128, 512, 1024, 2048),
             'epochs': epochs,
             'dropout': (0, 0.5, 1),
             'padding': 'same',
             'metrics': ['accuracy'],
             'weight_regulizer': [None],
             'emb_output_dims': [None],
             'shape': ['brick', 'long_funnel'],
             'optimizer': ['adam', 'Nadam', 'RMSprop', SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)],
             'losses': [mse, logcosh, binary_crossentropy, categorical_crossentropy],
             'activation': [relu, elu, linear],
             'last_activation': [softmax, sigmoid],
             'nb_classes': 10}

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
def _random_param(param):
    """
    _random_param : do random to param values
    :param param:
    :return:
    """
    rand_param = {'input_shape': param['input_shape'],
                  'input_shape_cnn': param['input_shape_cnn'],
                  'input_shape_rnn': param['input_shape_rnn'],
                  'lr': param['lr'],
                  'hidden_dim': param['hidden_dim'],
                  'units': param['units'],
                  'unitsSlp': param['unitsSlp'],
                  'padding': param['padding'],
                  'last_units': param['last_units'],
                  'first_neuron': random.choice(param['first_neuron']),
                  'hidden_layers': random.choice(param['hidden_layers']),
                  'kernel_constraint': param['kernel_constraint'],
                  'batch_size': random.choice(param['batch_size']),
                  'epochs': param['epochs'],
                  'dropout': random.choice(param['dropout']),
                  'metrics': param['metrics'],
                  'weight_regulizer': param['weight_regulizer'],
                  'emb_output_dims': ['emb_output_dims'],
                  'shape': random.choice(param['shape']),
                  'optimizer': random.choice(param['optimizer']),
                  'losses': random.choice(param['losses']),
                  'activation': random.choice(param['activation']),
                  'last_activation': random.choice(param['last_activation']),
                  'nb_classes': param['nb_classes']}
    return rand_param

# II. Prétraitement de la donnée

In [7]:
def _preprocess_cifar10(param):
    """
    :param dataset:
    :return:
    """
    (__X_train, __y_train), (__X_test, __y_test) = cifar10.load_data()
    __X_train = __X_train.reshape(50000, 32 * 32 * 3)
    __X_test = __X_test.reshape(10000, 32 * 32 * 3)

    __X_train = __X_train.astype('float32')
    __X_test = __X_test.astype('float32')
    __X_train /= 255.0
    __X_test /= 255.0

    __y_train = np_utils.to_categorical(__y_train, param['nb_classes'])
    __y_test = np_utils.to_categorical(__y_test, param['nb_classes'])
    return (__X_train, __y_train), (__X_test, __y_test)

## 1. Charge et prépare le dataset

In [8]:
param = _random_param(Param)
dataset = _preprocess_cifar10(param)

170500096/170498071 [==============================] - 532s 3us/step


In [10]:
param

{'input_shape': 3072,
 'input_shape_cnn': (32, 32, 3),
 'input_shape_rnn': (32, 96),
 'lr': 0.007723198108458322,
 'hidden_dim': 128,
 'units': 512,
 'unitsSlp': 10,
 'padding': 'same',
 'last_units': 10,
 'first_neuron': 16,
 'hidden_layers': 9,
 'kernel_constraint': <keras.constraints.MaxNorm at 0x7fd3ca3db470>,
 'batch_size': 64,
 'epochs': 1,
 'dropout': 0.5,
 'metrics': ['accuracy'],
 'weight_regulizer': [None],
 'emb_output_dims': ['emb_output_dims'],
 'shape': 'long_funnel',
 'optimizer': 'RMSprop',
 'losses': <function keras.losses.binary_crossentropy(y_true, y_pred)>,
 'activation': <function keras.activations.elu(x, alpha=1.0)>,
 'last_activation': <function keras.activations.softmax(x, axis=-1)>,
 'nb_classes': 10}

In [11]:
# load data
(X_train, y_train), (X_test, y_test) = dataset
nb_classes = y_test.shape[1]

# III. Gestion du modèle

## 1. Declaration du modèle

In [13]:
    def run_model(param):
        '''
        :return:
        '''
        (X_train, y_train), (X_test, y_test) = dataset
        type_model = "slp"

        model = Sequential()
        model.add(Dense(param['unitsSlp'],
                        input_shape=(param['input_shape'],),
                        activation=param['activation']))

        model.compile(loss=param['losses'],
                      optimizer=param['optimizer'],
                      metrics=param['metrics'])
        model.summary()

        tb_callback = self.__save_tensorboard(model, type_model)
        # training
        history = model.fit(X_train, y_train,
                            batch_size=param['batch_size'],
                            epochs=param['epochs'],
                            verbose=1,
                            validation_data=(X_test, y_test))

        # Final evaluation of the model
        score = model.evaluate(X_test, y_test, verbose=1)
        
        return history, model

## 4. Entrainement le modèle

In [14]:
talos.Scan(X_test,y_test,
           model=run_model,
           params=param)

ZeroDivisionError: float division by zero